In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from tqdm import tqdm
import json
import random
import torch
import numpy as np
from transformers import AutoTokenizer, T5Config
from malaya.torch_model.t5 import T5ForSequenceClassification

/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
config = T5Config.from_pretrained('mesolitica/nanot5-small-malaysian-cased')
config.num_labels = 2
config.vocab = ['contradiction', 'entailment']

In [4]:
model = T5ForSequenceClassification.from_pretrained('mesolitica/nanot5-small-malaysian-cased', config = config)
_ = model.cuda()

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at mesolitica/nanot5-small-malaysian-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-small-malaysian-cased')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]
trainer = torch.optim.AdamW(trainable_parameters, lr = 2e-4)

In [8]:
train_X, train_Y = [], []
with open('shuffled-train.json') as fopen:
    for l in fopen:
        l = json.loads(l)
        train_X.append(l['src'])
        train_Y.append(l['label'])

In [9]:
test_X, test_Y = [], []
with open('shuffled-test.json') as fopen:
    for l in fopen:
        l = json.loads(l)
        test_X.append(l['src'])
        test_Y.append(l['label'])
        
len(test_X)

16037

In [10]:
batch_size = 8
epoch = 100

In [12]:
best_dev_acc = -np.inf
patient = 1
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_X), batch_size))
    losses = []
    for i in pbar:
        trainer.zero_grad()
        x = train_X[i: i + batch_size]
        y = np.array(train_Y[i: i + batch_size])
        
        padded = tokenizer(x, truncation = True, padding = True, return_tensors = 'pt', max_length = 1024)
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
        
        padded.pop('token_type_ids', None)
            
        loss, pred = model(**padded)
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 1.0)
        trainer.step()
        losses.append(float(loss))
        pbar.set_postfix(loss = float(loss))
        
        
    dev_predicted = []
    for i in range(0, len(test_X[:10000]), batch_size):
        x = test_X[i: i + batch_size]
        y = np.array(test_Y[i: i + batch_size])
        padded = tokenizer(x, truncation = True, padding = True, return_tensors = 'pt', max_length = 1024)
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
            
        padded.pop('token_type_ids', None)
        
        loss, pred = model(**padded)
        dev_predicted.append((pred.argmax(axis = 1).detach().cpu().numpy() == y).mean())
        
    dev_predicted = np.mean(dev_predicted)
    
    print(f'epoch: {e}, loss: {np.mean(losses)}, dev_predicted: {dev_predicted}')
    
    if dev_predicted >= best_dev_acc:
        best_dev_acc = dev_predicted
        current_patient = 0
        model.save_pretrained('small')
    else:
        current_patient += 1
    
    if current_patient >= patient:
        break

100%|████████████████████| 136696/136696 [1:01:36<00:00, 36.98it/s, loss=0.0232]


epoch: 0, loss: 0.34205830785934066, dev_predicted: 0.8605


100%|███████████████████| 136696/136696 [1:01:39<00:00, 36.95it/s, loss=0.00923]


epoch: 1, loss: 0.27324160752283155, dev_predicted: 0.8619


100%|█████████████████████| 136696/136696 [1:01:40<00:00, 36.94it/s, loss=0.011]


epoch: 2, loss: 0.23670191810263938, dev_predicted: 0.8611


In [13]:
real_Y = []
for i in tqdm(range(0, len(test_X), batch_size)):
    x = test_X[i: i + batch_size]
    y = np.array(test_Y[i: i + batch_size])
    padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
    padded['labels'] = torch.from_numpy(y)
    for k in padded.keys():
        padded[k] = padded[k].cuda()
        
    padded.pop('token_type_ids', None)

    loss, pred = model(**padded)
    real_Y.extend(pred.argmax(axis = 1).detach().cpu().numpy().tolist())

100%|██████████████████████████████████████| 2005/2005 [00:08<00:00, 242.21it/s]


In [14]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, test_Y,
        digits = 5
    )
)

              precision    recall  f1-score   support

           0    0.86342   0.85215   0.85775      7812
           1    0.86129   0.87198   0.86660      8225

    accuracy                        0.86232     16037
   macro avg    0.86236   0.86206   0.86218     16037
weighted avg    0.86233   0.86232   0.86229     16037



In [15]:
tokenizer.model_input_names = ['input_ids', 'attention_mask']

In [16]:
tokenizer.push_to_hub('mesolitica/finetune-mnli-nanot5-small', safe_serialization = True)

CommitInfo(commit_url='https://huggingface.co/mesolitica/finetune-mnli-nanot5-small/commit/4243d919165719d7924063d04bb630728cc16033', commit_message='Upload tokenizer', commit_description='', oid='4243d919165719d7924063d04bb630728cc16033', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
model.push_to_hub('mesolitica/finetune-mnli-nanot5-small', safe_serialization = True)

model.safetensors:   0%|          | 0.00/148M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/finetune-mnli-nanot5-small/commit/d0851e2f22c547608bf6f611723dbb0f3fb6e722', commit_message='Upload T5ForSequenceClassification', commit_description='', oid='d0851e2f22c547608bf6f611723dbb0f3fb6e722', pr_url=None, pr_revision=None, pr_num=None)